# Checking for convergenece of Train and Val loss

In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from samples.idvscript import main
from samples import ivddataset

# Path to trained weights file
COCO_WEIGHTS_PATH = "mask_rcnn_idv_0050.h5"

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Experiment Configuration

In [3]:
class IDVConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "idv"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50
    
    #RPN_ANCHOR_SCALES = (4,8,32, 64, 128)
    
    # Loss weights for more precise optimization.
    # Can be used for R-CNN training setup.
    LOSS_WEIGHTS = {
        "rpn_class_loss": 0.,
        "rpn_bbox_loss": 0.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }

    # Skip detections with < 60% confidence
    DETECTION_MIN_CONFIDENCE = 0.6

config = IDVConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.6
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.0, 'rpn_bbox_loss': 0.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

# Instantiate Model

In [5]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

# Load pretrained weights
model_path = COCO_WEIGHTS_PATH
model.load_weights(model_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
          "mrcnn_bbox", "mrcnn_mask"])

# Load Training and Validation Dataset

In [6]:
# Load training dataset
dataset_train = ivddataset.IDVDataset()
dataset_train.load_idv(dataset_dir=os.path.join("train"))
dataset_train.prepare()

# Validation dataset
dataset_val = ivddataset.IDVDataset()
dataset_val.load_idv(dataset_dir=os.path.join("val"))
dataset_val.prepare()

# Train

In [7]:
# *** This training schedule is an example. Update to your needs ***

# Training - Stage 1
print("Training network heads")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=3,
            layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=6,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=10,
            layers='all')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /volumes/notebooks/Final IDV/logs/idv20180622T1755/mask_rcnn_idv_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (Tim

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/3
49/50 [============================>.] - ETA: 2s - loss: 0.8441 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1174 - mrcnn_bbox_loss: 0.3681 - mrcnn_mask_loss: 0.3587

/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


50/50 [==============================] - 204s 4s/step - loss: 0.8325 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.1163 - mrcnn_bbox_loss: 0.3633 - mrcnn_mask_loss: 0.3529 - val_loss: 0.5500 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0362 - val_mrcnn_bbox_loss: 0.2675 - val_mrcnn_mask_loss: 0.2462
Epoch 2/3
50/50 [==============================] - 175s 4s/step - loss: 0.2733 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0286 - mrcnn_bbox_loss: 0.1469 - mrcnn_mask_loss: 0.0978 - val_loss: 0.5227 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0307 - val_mrcnn_bbox_loss: 0.2487 - val_mrcnn_mask_loss: 0.2433
Epoch 3/3
50/50 [==============================] - 175s 3s/step - loss: 0.2095 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0241 - mrcnn_bbox_loss: 0.1039 - mrcnn_mask_loss: 0.0815 - val_loss: 0.4

Epoch 7/10
50/50 [==============================] - 278s 6s/step - loss: 0.1693 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0114 - mrcnn_bbox_loss: 0.0709 - mrcnn_mask_loss: 0.0870 - val_loss: 0.3606 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0146 - val_mrcnn_bbox_loss: 0.1427 - val_mrcnn_mask_loss: 0.2033
Epoch 8/10
50/50 [==============================] - 235s 5s/step - loss: 0.1343 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0139 - mrcnn_bbox_loss: 0.0487 - mrcnn_mask_loss: 0.0717 - val_loss: 0.4290 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0141 - val_mrcnn_bbox_loss: 0.1655 - val_mrcnn_mask_loss: 0.2495
Epoch 9/10
50/50 [==============================] - 235s 5s/step - loss: 0.1291 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0162 - mrcnn_bbox_loss: 0.0414 - mrcnn_mask_loss: 0.0714 - 